In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [8]:
def getWikiTable(year, columns):
    """
    Get top 100 Billboards songs per given year
    
    """
    
    url = url = f'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_{year}'
    request = requests.get(url)
    website = request.text

    soup = BeautifulSoup(website, features = "lxml")
    table = soup.find('table', {'class': 'wikitable'})

    columns = columns
    rows = len(table.findAll('tr'))
    
    col_names = list()
    for node in range(columns):
        col_names.append(', '.join(table.findAll('th')[node].text.split()))

    df = pd.DataFrame(index = range(rows))

    for column in range(columns):
        values = list()
        for each_row in range(rows):
            row = table.findAll('tr')[each_row].findAll('td')
            if len(row) == columns:
                value = ', '.join(row[column].text.split('[')[0].split())
                value = value.replace(',', '')
                values.append(value)
               
                df[f'{column}'] = pd.Series(values)
    df.columns = col_names
    df['Year'] = year
    return df

In [9]:
top_100_songs = pd.DataFrame()
for year in range(2000, 2020):
    out = getWikiTable(year, 3).iloc[:100]
    out['Title'] = [title.replace('"', '') for title in out['Title']]
    top_100_songs = pd.concat([top_100_songs, out])

In [10]:
top_100_songs

,No.,Title,Artist(s),Year,№
0,1,Breathe,Faith Hill,2000,NaN
1,2,Smooth,Santana featuring Rob Thomas,2000,NaN
2,3,Maria Maria,Santana featuring The Product G&B,2000,NaN
3,4,I Wanna Know,Joe,2000,NaN
4,5,Everything You Want,Vertical Horizon,2000,NaN
...,...,...,...,...,...
95,96,Eyes on You,Chase Rice,2019,NaN
96,97,All to Myself,Dan + Shay,2019,NaN
97,98,Boyfriend,Ariana Grande and Social House,2019,NaN
98,99,Walk Me Home,Pink,2019,NaN


In [11]:
spotify_client_id = 'e0e51ec7feeb45b89b165ae1007c535d'
spotify_client_secret = 'cf444557e7ad446ebd5c2d4cc6a781cf'
genius_client_access_token = 'fB4ldLe2jlKTIM7rNpHEFDozmXC-ipojy2wYZPS3rICuruFr7TH0Q9eED3CkewS8'

In [12]:
client_credentials_manager = SpotifyClientCredentials(spotify_client_id, spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [13]:
album_ids = list()
artist_ids = list()
track_ids = list()
for i, row in top_100_songs.iterrows():
    
    try:
        artist_name = row['Artist(s)'].split('featuring')[0]
        track_name = row['Title']
#         print(i, row['Year'], artist_name, track_name)

        match = sp.search(q='artist:' + artist_name + ' track:' + track_name, type='track')
        album_ids.append(match['tracks']['items'][0]['album']['id'])
        artist_ids.append(match['tracks']['items'][0]['artists'][0]['id'])
        track_ids.append(match['tracks']['items'][0]['id'])
    except Exception as e:
        print(e)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [14]:
albumsTable = pd.DataFrame()
artistsTable = pd.DataFrame()
tracksTable = pd.DataFrame()

In [15]:
n = 20
albums_info = sp.albums(album_ids[:n])
artists_info = sp.artists(artist_ids[:n])
tracks_info = sp.tracks(track_ids[:n])
# audio_analysis = sp.audio_analysis(track_id)
audio_analysis = list()
for song_id in track_ids[:n]:
    audio_analysis.append(sp.audio_analysis(song_id))
audio_features = sp.audio_features(track_ids[:n])

In [16]:
drop_cols = ['duration', 'codestring', 'code_version', 'echoprintstring', 
             'echoprint_version', 'synchstring', 'synch_version', 
             'rhythmstring', 'rhythm_version']

for aa in audio_analysis:
    for key in drop_cols:
        del aa['track'][key]
    
for af in audio_features:
    for key in ['analysis_url', 'uri']:
        del af[key]

In [17]:
for i in range(len(albums_info['albums'])):
    
    album_info = albums_info['albums'][i]
    album_artists = list()
    list_of_artists = album_info['artists']
    for artist in list_of_artists:
        album_artists.append(artist['name'])

    album_tracks = list()
    for track in album_info['tracks']['items']:
        album_tracks.append(track['id'])
    
    album = pd.DataFrame()
    album.at['album_id', 0] = album_info['id']
    album.at['album_name', 0] = album_info['name']
    album.at['album_type', 0] = album_info['type']
    album.at['number_of_tracks', 0] = album_info['total_tracks']
    album.at['album_artists', 0] = album_artists
    album.at['track_ids', 0] = album_tracks
    album.at['album_popularity', 0] = album_info['popularity']
    album.at['album_available_markets', 0] = album_info['available_markets']
    album.at['album_href', 0] = album_info['href']
    album.at['album_genres', 0] = album_info['genres']
    album.at['album_label', 0] = album_info['label']
    album.at['album_images', 0] = album_info['images']
    album.at['album_release_date', 0] = album_info['release_date']
    album = album.transpose()
    albumsTable = pd.concat([albumsTable, album])

    artist_info = artists_info['artists'][i] 
    artist = pd.DataFrame()
    artist.at['artist_id', 0] = artist_info['id']
    artist.at['artist_name', 0] = artist_info['name']
    artist.at['artist_followers', 0] = artist_info['followers']['total']
    artist.at['artist_popularity', 0] = artist_info['popularity']
    artist.at['artist_genres', 0] = artist_info['genres']
    artist.at['artist_href', 0] = artist_info['href']
    artist.at['artist_images', 0] = artist_info['images']
    artist.at['artist_type', 0] = artist_info['type']
    artist = artist.transpose()
    artistsTable = pd.concat([artistsTable, artist])
    
    track_info = tracks_info['tracks'][i]
    featuring_artists = list()
    for artist in track_info['artists']:
        if artist['name'] != artist_info['name']:
            featuring_artists.append(artist['name'])

    track_information = pd.DataFrame()
    if featuring_artists:
        track_information.at['featuring_artists', 0] = featuring_artists
    else:
        track_information.at['featuring_artists', 0] = None
        
    track_information.at['track_name', 0] = track_info['name']
    track_information.at['track_popularity', 0] = track_info['popularity']
    track_information.at['available_markets', 0] = track_info['available_markets']
    
    aa = pd.DataFrame.from_dict(audio_analysis[i]['track'], orient = 'index')
    af = pd.DataFrame.from_dict(audio_features[i], orient = 'index')

    track = pd.concat([aa, af, track_information]).transpose()
    tracksTable = pd.concat([tracksTable, track])

In [18]:
albumsTable

,album_id,album_name,album_type,number_of_tracks,album_artists,track_ids,album_popularity,album_available_markets,album_href,album_genres,album_label,album_images,album_release_date
0,3BGPil8BDPVoDK0lnUPtvb,Breathe,album,13,[Faith Hill],"[05nzQRqCZ36DTDCbUb25Qa, 52UkT2gKZEYlEEKoXl6OD...",63,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/3BGPil8BDPVo...,[],Warner Records,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1999-11-09
0,10aiDpdFGyfCFEcqpx6XTq,Supernatural (Remastered),album,14,[Santana],"[3ZJMi7jX3j34ORbXFjrzQi, 7qtpPSLeZMGv0vrCZu5kx...",69,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/10aiDpdFGyfC...,[],Columbia/Legacy,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1999-06-15
0,10aiDpdFGyfCFEcqpx6XTq,Supernatural (Remastered),album,14,[Santana],"[3ZJMi7jX3j34ORbXFjrzQi, 7qtpPSLeZMGv0vrCZu5kx...",69,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/10aiDpdFGyfC...,[],Columbia/Legacy,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1999-06-15
0,6zuCJaxmHKNKN5hMDF556U,My Name Is Joe,album,14,[Joe],"[4PYnlh05Q2SobrLl8hglbf, 6ULFbCM2KhUgHDNzbR8DZ...",64,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/6zuCJaxmHKNK...,[],Jive,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2000-04-18
0,2gJ6axVHAsQmmO6nslQRN1,Everything You Want,album,11,[Vertical Horizon],"[3guj8JHApcuOZxsM9ZSsdg, 758wp6xs3ADdgIwdeHoXm...",62,"[AD, AE, AL, AR, AU, BA, BE, BG, BH, BO, BR, B...",https://api.spotify.com/v1/albums/2gJ6axVHAsQm...,[],RCA Records Label,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1999-06-14
0,283NWqNsCA9GwVHrJk59CG,The Writing's On The Wall,album,16,[Destiny's Child],"[0EnGScOVBSbIZDEzpRZzFa, 1pLJSpfeHXsIyJS1BmKrr...",74,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/283NWqNsCA9G...,[],Columbia,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1999-07-27
0,3w69KK7uEA8OsKuW3OeQcC,Affirmation,album,12,[Savage Garden],"[6lWY4ji9T0tpFU2xbnRC1p, 5z7UgjxvvKBTf3qZMDvdd...",67,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/3w69KK7uEA8O...,[],Columbia,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1999-11-09
0,7ykiOoPC4GLrVYSDVdDGq2,Lonely Grill,album,12,[Lonestar],"[23RXKgNdklPIzXMjmmL8vb, 5zhbTlQQ6GIorr4VyRUdj...",62,"[AD, AE, AL, AR, AU, BA, BE, BG, BH, BO, BR, B...",https://api.spotify.com/v1/albums/7ykiOoPC4GLr...,[],BNA Records Label,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1999
0,2HqQR5SkxWX7uUWaxlLksn,Mad Season,album,13,[Matchbox Twenty],"[356SS2KraT3fSKZRYCuKVX, 0CKefyZmTxVJLnGZGYwsD...",62,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/2HqQR5SkxWX7...,[],Atlantic Records,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2000-05-23
0,0UZsKcXzOehMvFWTiBlwMi,The Heat,album,12,[Toni Braxton],"[7f1Dmr246cJ9uQYdbplTbh, 7HErma1ht0kUE4kIahqrj...",64,"[AD, AE, AL, AR, AU, BA, BE, BG, BH, BO, BR, C...",https://api.spotify.com/v1/albums/0UZsKcXzOehM...,[],Arista/LaFace Records,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2000


In [19]:
artistsTable

,artist_id,artist_name,artist_followers,artist_popularity,artist_genres,artist_href,artist_images,artist_type
0,25NQNriVT2YbSW80ILRWJa,Faith Hill,1213140,69,"[contemporary country, country, country dawn, ...",https://api.spotify.com/v1/artists/25NQNriVT2Y...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,6GI52t8N5F02MxU0g5U69P,Santana,1777156,75,"[blues rock, classic rock, mexican classic roc...",https://api.spotify.com/v1/artists/6GI52t8N5F0...,"[{'height': 1000, 'url': 'https://i.scdn.co/im...",artist
0,6GI52t8N5F02MxU0g5U69P,Santana,1777156,75,"[blues rock, classic rock, mexican classic roc...",https://api.spotify.com/v1/artists/6GI52t8N5F0...,"[{'height': 1000, 'url': 'https://i.scdn.co/im...",artist
0,3zTOe1BtyTkwNvYZOxXktX,Joe,1117792,69,"[dance pop, hip pop, neo soul, new jack swing,...",https://api.spotify.com/v1/artists/3zTOe1BtyTk...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,6Hizgjo92FnMp8wGaRUNTn,Vertical Horizon,299006,57,"[neo mellow, pop rock, post-grunge]",https://api.spotify.com/v1/artists/6Hizgjo92Fn...,"[{'height': 1000, 'url': 'https://i.scdn.co/im...",artist
0,1Y8cdNmUJH7yBTd9yOvr5i,Destiny's Child,4400685,76,"[dance pop, girl group, hip pop, pop, r&b, urb...",https://api.spotify.com/v1/artists/1Y8cdNmUJH7...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,3NRFinRTEqUCfaTTZmk8ek,Savage Garden,765527,69,"[australian pop, boy band, dance pop, europop,...",https://api.spotify.com/v1/artists/3NRFinRTEqU...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,3qbnxnvUqR14MJ9g8QwZJK,Lonestar,917566,64,"[contemporary country, country, country road]",https://api.spotify.com/v1/artists/3qbnxnvUqR1...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,3Ngh2zDBRPEriyxQDAMKd1,Matchbox Twenty,1701625,71,"[neo mellow, pop rock, post-grunge]",https://api.spotify.com/v1/artists/3Ngh2zDBRPE...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,3X458ddYA2YcVWuVIGGOYe,Toni Braxton,2364044,70,"[dance pop, hip pop, neo soul, new jack swing,...",https://api.spotify.com/v1/artists/3X458ddYA2Y...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist


In [20]:
tracksTable

,num_samples,sample_md5,offset_seconds,window_seconds,analysis_sample_rate,analysis_channels,end_of_fade_in,start_of_fade_out,loudness,tempo,...,tempo,type,id,track_href,duration_ms,time_signature,featuring_artists,track_name,track_popularity,available_markets
0,5524554,,0,0,22050,1,0.26005,237.894,-9.007,136.859,...,136.859,audio_features,3y4LxiYMgDl4RethdzpmNe,https://api.spotify.com/v1/tracks/3y4LxiYMgDl4...,250547,4,NaN,Breathe,67,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,6504456,,0,0,22050,1,0.19138,281.821,-3.908,115.996,...,115.996,audio_features,0n2SEXB2qoRQg171q7XqeW,https://api.spotify.com/v1/tracks/0n2SEXB2qoRQ...,294987,4,Rob Thomas,Smooth (feat. Rob Thomas),69,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,5776512,,0,0,22050,1,0.22544,252.244,-5.931,97.911,...,97.911,audio_features,3XKIUb7HzIF1Vu9usunMzc,https://api.spotify.com/v1/tracks/3XKIUb7HzIF1...,261973,4,The Product G&B,Maria Maria (feat. The Product G&B),67,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,6542088,,0,0,22050,1,2.0898,283.417,-5.959,136.086,...,136.086,audio_features,1m2xMsxbtxv21Brome189p,https://api.spotify.com/v1/tracks/1m2xMsxbtxv2...,296693,4,NaN,I Wanna Know,66,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,5668320,,0,0,22050,1,0.3356,243.798,-6.389,93.898,...,93.898,audio_features,4cKGldbhGJniI8BrB3K6tb,https://api.spotify.com/v1/tracks/4cKGldbhGJni...,257067,4,NaN,Everything You Want,64,"[AD, AE, AL, AR, AU, BA, BE, BG, BH, BO, BR, B..."
0,5982900,,0,0,22050,1,0.14494,265.375,-3.525,138.009,...,138.009,audio_features,7H6ev70Weq6DdpZyyTmUXk,https://api.spotify.com/v1/tracks/7H6ev70Weq6D...,271333,4,NaN,Say My Name,77,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,5520438,,0,0,22050,1,0.08698,233.761,-9.88,85.031,...,85.031,audio_features,6nozDLxeL0TE4MS9GqYU1v,https://api.spotify.com/v1/tracks/6nozDLxeL0TE...,250360,4,NaN,I Knew I Loved You,70,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,5311110,,0,0,22050,1,0.26717,229.251,-8.498,139.803,...,139.803,audio_features,6qc34bnVOyqGDPni8H5W0U,https://api.spotify.com/v1/tracks/6qc34bnVOyqG...,240867,4,NaN,Amazed,69,"[AD, AE, AL, AR, AU, BA, BE, BG, BH, BO, BR, B..."
0,5647740,,0,0,22050,1,0,242.817,-6.814,95.468,...,95.468,audio_features,6fA7akEuTUL3dW1V0GELaZ,https://api.spotify.com/v1/tracks/6fA7akEuTUL3...,256133,4,NaN,Bent,56,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,5775630,,0,0,22050,1,0.33655,259.268,-1.915,88.009,...,88.009,audio_features,7f1Dmr246cJ9uQYdbplTbh,https://api.spotify.com/v1/tracks/7f1Dmr246cJ9...,261933,4,NaN,He Wasn't Man Enough,68,"[AD, AE, AL, AR, AU, BA, BE, BG, BH, BO, BR, C..."


In [21]:
genius = lyricsgenius.Genius(genius_client_access_token)

In [22]:
lyrics_table = {}
for i, row in top_100_songs.iterrows():
    
    try:
        artist_name = row['Artist(s)'].split('featuring')[0]
        track_name = row['Title']
        print(i, row['Year'], artist_name, track_name)

        lyrics_match = genius.search_song(track_name, artist_name)
        lyrics_table[artist_name] = {track_name:lyrics_match.lyrics}
        
    except Exception as e:
        print(e)
import pickle
pickle.dump(lyrics_table, open('lyrics_table.pkl', 'wb'))

0 2000 Faith Hill Breathe
Searching for "Breathe" by Faith Hill...
Done.
1 2000 Santana  Smooth
Searching for "Smooth" by Santana ...
Done.
2 2000 Santana  Maria Maria
Searching for "Maria Maria" by Santana ...
Done.
3 2000 Joe I Wanna Know
Searching for "I Wanna Know" by Joe...
Done.
4 2000 Vertical Horizon Everything You Want
Searching for "Everything You Want" by Vertical Horizon...
Done.
5 2000 Destiny's Child Say My Name
Searching for "Say My Name" by Destiny's Child...
Done.
6 2000 Savage Garden I Knew I Loved You
Searching for "I Knew I Loved You" by Savage Garden...
Done.
7 2000 Lonestar Amazed
Searching for "Amazed" by Lonestar...
Done.
8 2000 Matchbox Twenty Bent
Searching for "Bent" by Matchbox Twenty...
Done.
9 2000 Toni Braxton He Wasn't Man Enough
Searching for "He Wasn't Man Enough" by Toni Braxton...
Done.
10 2000 Creed Higher
Searching for "Higher" by Creed...
Done.
11 2000 Aaliyah Try Again
Searching for "Try Again" by Aaliyah...
Done.
12 2000 Destiny's Child Jumpin' 

Done.
87 2000 Alice Deejay Better Off Alone
Searching for "Better Off Alone" by Alice Deejay...
Done.
88 2000 Debelah Morgan Dance with Me
Searching for "Dance with Me" by Debelah Morgan...
Done.
89 2000 Lonestar What About Now
Searching for "What About Now" by Lonestar...
Done.
90 2000 Sammie I Like It
Searching for "I Like It" by Sammie...
Done.
91 2000 Kevon Edmonds 24/7
Searching for "24/7" by Kevon Edmonds...
Done.
92 2000 LFO Girl on TV
Searching for "Girl on TV" by LFO...
Done.
93 2000 Lil' Bow Wow  Bounce with Me
Searching for "Bounce with Me" by Lil' Bow Wow ...
Done.
94 2000 Dixie Chicks Cowboy Take Me Away
Searching for "Cowboy Take Me Away" by Dixie Chicks...
Done.
95 2000 Aaliyah I Don't Wanna
Searching for "I Don't Wanna" by Aaliyah...
Done.
96 2000 Destiny's Child Independent Women
Searching for "Independent Women" by Destiny's Child...
Done.
97 2000 Samantha Mumba Gotta Tell You
Searching for "Gotta Tell You" by Samantha Mumba...
Done.
98 2000 Jennifer Lopez Waiting for

'NoneType' object is not subscriptable
72 2001 Lonestar I'm Already There
Searching for "I'm Already There" by Lonestar...
Done.
73 2001 Lil' Romeo My Baby
Searching for "My Baby" by Lil' Romeo...
Done.
74 2001 U2 Beautiful Day
Searching for "Beautiful Day" by U2...
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
'NoneType' object is not subscriptable
75 2001 Blake Shelton Austin
Searching for "Austin" by Blake Shelton...
Done.
76 2001 Ludacris Southern Hospitality
Searching for "Southern Hospitality" by Ludacris...
Done.
77 2001 Tim McGraw Grown Men Don't Cry
Searching for "Grown Men Don't Cry" by Tim McGraw...
Done.
78 2001 Ja Rule  Livin' It Up
Searching for "Livin' It Up" by Ja Rule ...
Done.
79 2001 Mariah Carey  Loverboy
Searching for "Loverboy" by Mariah Carey ...
Done.
80 2001 The Isley Brothers  Contagious
Searching for "Contagious" by The Isley Brothers ...
Done.
81 2001 Jessica Andrews Who I Am
Searching for "

Done.
51 2002 Craig David 7 Days
Searching for "7 Days" by Craig David...
Done.
52 2002 Cam'ron  Hey Ma
Searching for "Hey Ma" by Cam'ron ...
Done.
53 2002 Missy Elliott Work It
Searching for "Work It" by Missy Elliott...
Done.
54 2002 Ludacris  Move Bitch
Searching for "Move Bitch" by Ludacris ...
Done.
55 2002 LeAnn Rimes Can't Fight the Moonlight
Searching for "Can't Fight the Moonlight" by LeAnn Rimes...
Done.
56 2002 Enrique Iglesias Escape
Searching for "Escape" by Enrique Iglesias...
Done.
57 2002 Aaliyah More than a Woman
Searching for "More than a Woman" by Aaliyah...
Done.
58 2002 No Doubt Hella Good
Searching for "Hella Good" by No Doubt...
Done.
59 2002 Faith Evans I Love You
Searching for "I Love You" by Faith Evans...
Done.
60 2002 Daniel Bedingfield Gotta Get thru This
Searching for "Gotta Get thru This" by Daniel Bedingfield...
Done.
61 2002 Busta Rhymes  Pass the Courvoisier Part II
Searching for "Pass the Courvoisier Part II" by Busta Rhymes ...
Done.
62 2002 Eminem L

Done.
29 2003 Justin Timberlake Cry Me a River
Searching for "Cry Me a River" by Justin Timberlake...
Done.
30 2003 Tyrese How You Gonna Act Like That
Searching for "How You Gonna Act Like That" by Tyrese...
Done.
31 2003 Justin Timberlake Rock Your Body
Searching for "Rock Your Body" by Justin Timberlake...
Done.
32 2003 Wayne Wonder No Letting Go
Searching for "No Letting Go" by Wayne Wonder...
Done.
33 2003 Pharrell  Frontin'
Searching for "Frontin'" by Pharrell ...
Timeout raised and caught:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)
'NoneType' object is not subscriptable
34 2003 Dixie Chicks Landslide
Searching for "Landslide" by Dixie Chicks...
Done.
35 2003 Missy Elliott Work It
Searching for "Work It" by Missy Elliott...
Done.
36 2003 Jay-Z  '03 Bonnie & Clyde
Searching for "'03 Bonnie & Clyde" by Jay-Z ...
Done.
37 2003 Nivea  Don't Mess with My Man
Searching for "Don't Mess with My Man" by Nivea ...
Timeout raised and caught:
HTTPSConne

...

Done.
66 2015 Ariana Grande One Last Time
Searching for "One Last Time" by Ariana Grande...
Done.
67 2015 Nick Jonas Chains
Searching for "Chains" by Nick Jonas...
Done.
68 2015 Meek Mill  All Eyes on You
Searching for "All Eyes on You" by Meek Mill ...
Done.
69 2015 Rae Sremmurd No Type
Searching for "No Type" by Rae Sremmurd...
Done.
70 2015 Vance Joy Riptide
Searching for "Riptide" by Vance Joy...
Done.
71 2015 Jidenna  Classic Man
Searching for "Classic Man" by Jidenna ...
Done.
72 2015 Elle King Ex's & Oh's
Searching for "Ex's & Oh's" by Elle King...
Done.
73 2015 Meghan Trainor Dear Future Husband
Searching for "Dear Future Husband" by Meghan Trainor...
Done.
74 2015 Charlie Puth  Marvin Gaye
Searching for "Marvin Gaye" by Charlie Puth ...
Done.
75 2015 Meghan Trainor  Like I'm Gonna Lose You
Searching for "Like I'm Gonna Lose You" by Meghan Trainor ...
Done.
76 2015 Tove Lo Habits (Stay High)
Searching for "Habits (Stay High)" by Tove Lo...
Done.
77 2015 James Newton Howard  The

Done.
61 2016 Meghan Trainor Me Too
Searching for "Me Too" by Meghan Trainor...
Done.
62 2016 Elle King Ex's & Oh's
Searching for "Ex's & Oh's" by Elle King...
Done.
63 2016 Thomas Rhett Die a Happy Man
Searching for "Die a Happy Man" by Thomas Rhett...
Done.
64 2016 Post Malone White Iverson
Searching for "White Iverson" by Post Malone...
Done.
65 2016 Nick Jonas  Close
Searching for "Close" by Nick Jonas ...
Done.
66 2016 X Ambassadors Unsteady
Searching for "Unsteady" by X Ambassadors...
Done.
67 2016 Lil Wayne Wiz Khalifa and Imagine Dragons with Logic and Ty Dolla $ign  Sucker for Pain
Searching for "Sucker for Pain" by Lil Wayne Wiz Khalifa and Imagine Dragons with Logic and Ty Dolla $ign ...
Done.
68 2016 Yo Gotti  Down in the DM
Searching for "Down in the DM" by Yo Gotti ...
Done.
69 2016 Tory Lanez Luv
Searching for "Luv" by Tory Lanez...
Done.
70 2016 Beyoncé Sorry
Searching for "Sorry" by Beyoncé...
Done.
71 2016 The Weeknd Can't Feel My Face
Searching for "Can't Feel My Fac

Done.
50 2017 Imagine Dragons Thunder
Searching for "Thunder" by Imagine Dragons...
Done.
51 2017 Migos T-Shirt
Searching for "T-Shirt" by Migos...
Done.
52 2017 Yo Gotti  Rake It Up
Searching for "Rake It Up" by Yo Gotti ...
Done.
53 2017 Shawn Mendes Mercy
Searching for "Mercy" by Shawn Mendes...
Done.
54 2017 Kodak Black Tunnel Vision
Searching for "Tunnel Vision" by Kodak Black...
Done.
55 2017 Post Malone  Rockstar
Searching for "Rockstar" by Post Malone ...
Done.
56 2017 Brett Young In Case You Didn't Know
Searching for "In Case You Didn't Know" by Brett Young...
Done.
57 2017 Twenty One Pilots Heathens
Searching for "Heathens" by Twenty One Pilots...
Done.
58 2017 Halsey Now or Never
Searching for "Now or Never" by Halsey...
Done.
59 2017 Aminé Caroline
Searching for "Caroline" by Aminé...
Done.
60 2017 Ayo & Teo Rolex
Searching for "Rolex" by Ayo & Teo...
Done.
61 2017 Kendrick Lamar DNA
Searching for "DNA" by Kendrick Lamar...
Done.
62 2017 Zay Hilfigerrr & Zayion McCall Juju 

Done.
44 2018 G-Eazy and Halsey Him & I
Searching for "Him & I" by G-Eazy and Halsey...
Done.
45 2018 Shawn Mendes In My Blood
Searching for "In My Blood" by Shawn Mendes...
Done.
46 2018 Kendrick Lamar and SZA All the Stars
Searching for "All the Stars" by Kendrick Lamar and SZA...
Done.
47 2018 Migos Stir Fry
Searching for "Stir Fry" by Migos...
Done.
48 2018 Sam Smith Too Good at Goodbyes
Searching for "Too Good at Goodbyes" by Sam Smith...
Done.
49 2018 Kendrick Lamar  Love
Searching for "Love" by Kendrick Lamar ...
Done.
50 2018 Childish Gambino This Is America
Searching for "This Is America" by Childish Gambino...
Done.
51 2018 Drake Nonstop
Searching for "Nonstop" by Drake...
Done.
52 2018 Kane Brown Heaven
Searching for "Heaven" by Kane Brown...
Done.
53 2018 Cardi B Bodak Yellow
Searching for "Bodak Yellow" by Cardi B...
Done.
54 2018 Lil Dicky  Freaky Friday
Searching for "Freaky Friday" by Lil Dicky ...
Done.
55 2018 6ix9ine Gummo
Searching for "Gummo" by 6ix9ine...
Done.
56

Done.
38 2019 Taylor Swift You Need to Calm Down
Searching for "You Need to Calm Down" by Taylor Swift...
Done.
39 2019 Lil Nas X Panini
Searching for "Panini" by Lil Nas X...
Done.
40 2019 A Boogie wit da Hoodie Look Back at It
Searching for "Look Back at It" by A Boogie wit da Hoodie...
Done.
41 2019 21 Savage A Lot
Searching for "A Lot" by 21 Savage...
Done.
42 2019 Taylor Swift  Me!
Searching for "Me!" by Taylor Swift ...
Done.
43 2019 Bad Bunny  Mia
Searching for "Mia" by Bad Bunny ...
Done.
44 2019 Polo G  Pop Out
Searching for "Pop Out" by Polo G ...
Done.
45 2019 Luke Combs Beautiful Crazy
Searching for "Beautiful Crazy" by Luke Combs...
Done.
46 2019 Blueface Thotiana
Searching for "Thotiana" by Blueface...
Done.
47 2019 Juice Wrld Lucid Dreams
Searching for "Lucid Dreams" by Juice Wrld...
Done.
48 2019 Sheck Wes Mo Bamba
Searching for "Mo Bamba" by Sheck Wes...
Done.
49 2019 Ed Sheeran  Beautiful People
Searching for "Beautiful People" by Ed Sheeran ...
Done.
50 2019 Gucci Ma

In [23]:
lyrics_table

{'Faith Hill': {'Mississippi Girl': "[Intro]\nWell, it's a long way from Star, Mississippi\nTo the big stage I'm singing on tonight\nAnd sometimes the butterflies still get me\nWhen I'm in the spotlight\n\n[Verse 1]\nAnd some people seem to think that I've changed\nThat I'm different than I was back then\nBut in my soul, I know that I'm the same way\nThat I've really always been\n\n[Chorus]\n'Cause a Mississippi girl don't change her ways\nJust 'cause everybody knows her name\nAin't big-headed from a little bit of fame\nI still like wearing my old ball cap\nRide my kids around piggy back\nThey might know me all around the world\nBut y'all, I'm still a Mississippi girl\n\n[Verse 2]\nWell, I spent a few weeks in California\nThey put my face on the big movie screen\nBut that don't mean I've forgotten where I came from\nThat's just me chasing dreams, yeah\n\n[Chorus]\n'Cause a Mississippi girl don't change her ways\nJust 'cause everybody knows her name\nAin't big-headed from a little bit o

In [24]:
lyrics_match = genius.search_song('Breathe', 'Faith Hill') #Breathe	Faith Hill

Searching for "Breathe" by Faith Hill...
Done.


In [25]:
lyrics_match.lyrics

"[Intro]\nI can feel the magic floating in the air\nBeing with you gets me that way\nI watch the sunlight dance across your face and I've\nNever been this swept away\n\n[Verse 1]\nAll my thoughts just seem to settle on the breeze\nWhen I'm lying wrapped up in your arms\nThe whole world just fades away\nThe only thing I hear\nIs the beating of your heart\n\n[Chorus]\n'Cause I can feel you breathe, it's washing over me\nAnd suddenly, I'm melting into you\nThere's nothing left to prove\nBaby, all we need is just to be\nCaught up in the touch, slow and steady rush\nBaby, isn't that the way that love's supposed to be?\nI can feel you breathe\nJust breathe\n\n[Verse 2]\nIn a way, I know my heart is waking up\nAs all the walls come tumbling down\nI'm closer than I've ever felt before\nAnd I know, and you know\nThere's no need for words right now\n\n[Chorus]\n'Cause I can feel you breathe, it's washing over me\nAnd suddenly, I'm melting into you\nThere's nothing left to prove\nBaby, all we nee

In [26]:
lyrics_match.writer_artists

[{'api_path': '/artists/551029',
  'header_image_url': 'https://assets.genius.com/images/default_avatar_300.png?1605635080',
  'id': 551029,
  'image_url': 'https://assets.genius.com/images/default_avatar_300.png?1605635080',
  'is_meme_verified': False,
  'is_verified': False,
  'name': 'Holly Lamar',
  'url': 'https://genius.com/artists/Holly-lamar'},
 {'api_path': '/artists/344559',
  'header_image_url': 'https://images.genius.com/0e9c7aa37d3b951756ce792a19e8000c.400x400x1.jpg',
  'id': 344559,
  'image_url': 'https://images.genius.com/0e9c7aa37d3b951756ce792a19e8000c.400x400x1.jpg',
  'is_meme_verified': False,
  'is_verified': False,
  'name': 'Stephanie Bentley',
  'url': 'https://genius.com/artists/Stephanie-bentley'}]

In [27]:
lyrics_match.producer_artists

[{'api_path': '/artists/48448',
  'header_image_url': 'https://images.genius.com/41fef630e8b5c78b62920d4d9b636d65.570x570x1.jpg',
  'id': 48448,
  'image_url': 'https://images.genius.com/41fef630e8b5c78b62920d4d9b636d65.570x570x1.jpg',
  'is_meme_verified': False,
  'is_verified': False,
  'name': 'Faith Hill',
  'url': 'https://genius.com/artists/Faith-hill'},
 {'api_path': '/artists/96793',
  'header_image_url': 'https://images.genius.com/1801ffac1b72971cff7553fc253e2ce5.937x1000x1.jpg',
  'id': 96793,
  'image_url': 'https://images.genius.com/1801ffac1b72971cff7553fc253e2ce5.937x1000x1.jpg',
  'is_meme_verified': False,
  'is_verified': False,
  'name': 'Byron Gallimore',
  'url': 'https://genius.com/artists/Byron-gallimore'}]

In [28]:
lyrics_match.media

[{'native_uri': 'spotify:track:3y4LxiYMgDl4RethdzpmNe',
  'provider': 'spotify',
  'type': 'audio',
  'url': 'https://open.spotify.com/track/3y4LxiYMgDl4RethdzpmNe'},
 {'provider': 'youtube',
  'start': 0,
  'type': 'video',
  'url': 'http://www.youtube.com/watch?v=yCmsZUN4r_s'}]